# Derived Features using ScikitLearn Transformers

Derived features are features used by the models that are computed from other features. We compute these derived features using the [ScikitLearn](https://scikit-learn.org/stable/) Pipelines and Transformers.

You can check all the implemented transformers in the `feature_store/derived_features/` directory.

In [1]:
%load_ext autoreload
%autoreload 2

%env LOG_LEVEL=WARNING

env: LOG_LEVEL=WARNING


In [2]:
import os

import pandas as pd
from pyprojroot import here

## Extract Example Features

In [3]:
from data_access_layer.dal import SklearnDataAccessLayer
from feature_store.feature_views import DiabetesFeatureView

In [4]:
dal = SklearnDataAccessLayer().connect()

In [5]:
data = DiabetesFeatureView.read(dal)
data.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


## Use Transformers Independently

You can use the derived features transformers just as they are. This is useful for debuging or get derived data quickly for analysis.

In [6]:
from feature_store.derived_features import row_features

In [7]:
t = row_features.FormulaTransformer(formula="bmi_sex_interaction = bmi * sex")
t

FormulaTransformer(formula='bmi_sex_interaction = bmi * sex')

In [8]:
t.fit_transform(data).head()

,bmi_sex_interaction
0,0.003127
1,0.002298
2,0.002253
3,0.000518
4,0.001624


## Create Feature Engineering Pipeline

It is useful to compose different transformers in a pipeline to be able to use them in the model pipeline. We need to use sklearn `ColumnTransformer` and `Pipeline` to achive this.

In [9]:
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline

In [10]:
pipeline = make_pipeline(
    make_column_transformer(
        (
            row_features.FormulaTransformer(formula="bmi_sex_interaction = bmi * sex"),
            ["bmi", "sex"],
        ),
        (
            row_features.FormulaTransformer(formula="s1_s2_normalized = s1 / s2"),
            ["s1", "s2"],
        ),
        # ("passthrough", make_column_selector()),
        verbose_feature_names_out=False,
        remainder="passthrough",
    )
)
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('formulatransformer-1',
                                                  FormulaTransformer(formula='bmi_sex_interaction '
                                                                             '= '
                                                                             'bmi '
                                                                             '* '
                                                                             'sex'),
                                                  ['bmi', 'sex']),
                                                 ('formulatransformer-2',
                                                  FormulaTransformer(formula='s1_s2_normalized '
                                                                             '= '
                                                                             's1 '
                                                                             '/ '
                                                                             's2'),
                                                  ['s1', 's2'])],
                                   verbose_feature_names_out=False))])

In [11]:
pipeline.fit_transform(data).head()

,bmi_sex_interaction,s1_s2_normalized,x0,x3,x6,x7,x8,x9,x10
0,0.003127,1.270032,0.038076,0.021872,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,0.002298,0.440880,-0.001882,-0.026328,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.002253,1.333533,0.085299,-0.005670,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,0.000518,0.487806,-0.089063,-0.036656,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.001624,0.252297,0.005383,0.021872,0.008142,-0.002592,-0.031988,-0.046641,135.0


Using the `passthrough` option, we get the union of the original features plus the derived features.

---